In [286]:
#https://medium.com/dida-machine-learning/inside-x-rocket-explaining-the-explainable-rocket-534b104c4a08

In [98]:
# https://www.sktime.net/en/v0.24.1/examples/transformation/rocket.html
from sktime.datatypes import check_raise, convert_to
from sktime.transformations.panel.rocket import Rocket
from matplotlib import pyplot as plt
from sktime.utils.plotting import plot_series
from sklearn.linear_model import RidgeClassifierCV, RidgeCV
import ipywidgets as widgets
import numpy as np
import pandas as pd
import os
from sktime.pipeline import make_pipeline

In [99]:
#import ipywidgets as widgets
#from ipywidgets import interact, interact_manual

In [100]:
# Load all the dataframes for either test or training
# Ensure all their missing columns can be filled in with zeros
# Make a list of them, use check_raise and the df-list mtype to ensure they are valid
# Pass to ROCKET pipeline as seen

In [101]:
def load_invidual_instance(filename, needed_columns):
    df = pd.read_csv(filename)
    for col in needed_columns:
         if not (col in df.columns):
            df[col] = 0.0
    return df

In [102]:
def create_combined_data(base_dir, filenames, needed_columns):
    combined_data_m = map(lambda file: load_invidual_instance(base_dir + "/" + file, needed_columns), filenames)
    combined_data = list(combined_data_m)
    print("Check data: ",check_raise(combined_data, mtype="df-list"))
    return combined_data

In [103]:
needed_columns = ['dosAttackTrigger', 'distortBaseScanPMB2', 'distortBaseScanOmni', 'distortBaseScanTiago', 'packetLossLaserScanOmni', 'packetLossLaserScanPMB2', 'packetLossLaserScanTiago', 'delayBaseScanOmni', 'delayBaseScanPMB2', 'delayBaseScanTiago1']
create_combined_data("/home/jharbin/academic/soprano/SPworkerTemp/notebooks/predictor/temp-data/train", ["Test_001_18_01_2024_05_28_25.csv"], needed_columns)


FileNotFoundError: [Errno 2] No such file or directory: '/home/jharbin/academic/soprano/SPworkerTemp/notebooks/predictor/temp-data/train/Test_001_18_01_2024_05_28_25.csv'

In [104]:
def find_data_in_dir(csv_directory_train, csv_directory_test, train_split_num):
    files_train = os.listdir(csv_directory_train)
    files_test = os.listdir(csv_directory_test)
    return files_train, files_test

In [105]:
m = 3
np.floor(m / 2)

1.0

In [174]:
def single_metric_value(m):
    f = np.floor(m/2)
    if (f < 0):
        return 0.0
    else:
        return f

In [175]:
def metrics_map_to_class(metrics):
      return np.array([single_metric_value(v) for v in metrics])

In [176]:
dat = pd.read_csv("/home/jharbin/academic/soprano/SPWorkerTemp/notebooks/predictor/temp-data/metrics-train.csv")["M1_countObjectsMissed"]
metrics_map_to_class(dat)

array([1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 1., 2., 1., 0., 0., 0., 2., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0.

In [177]:
metric_map_to_class(np.arange(0,7))

[0.0, 0.0, 1.0, 1.0, 2.0, 2.0, 3.0]

In [178]:
def rocket_complete(csv_directory_train, csv_directory_test, train_mfile, test_mfile, train_split_num, needed_columns, num_kernels):
    files_train = os.listdir(csv_directory_train)
    files_test = os.listdir(csv_directory_test)
    train_data = create_combined_data(csv_directory_train, files_train, needed_columns)
    test_data = create_combined_data(csv_directory_test, files_test, needed_columns)
    train_metrics = pd.read_csv(train_mfile)
    test_metrics = pd.read_csv(test_mfile)
    train_class = metrics_map_to_class(train_metrics["M1_countObjectsMissed"])
    test_class = metric_map_to_class(test_metrics["M1_countObjectsMissed"])
    
    for d in train_data:
        print("Shape:", d.shape)
    rocket_pipeline_classifier = make_pipeline(Rocket(num_kernels=num_kernels), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))
    rocket_pipeline = make_pipeline(Rocket(num_kernels=num_kernels), RidgeCV(alphas=(0.1, 1.0, 10.0)))
    
    #rocket_pipeline.fit(train_data, train_metrics)
    #print("ROCKET fit done!")
    #score = rocket_pipeline.score(test_data, test_metrics)
    #print("Score on test data = ",score)
    #print("ROCKET score done!")
    
    rocket_pipeline_classifier.fit(train_data, train_class)
    print("ROCKET classifier fit done!")
    score = rocket_pipeline_classifier.score(test_data, test_class)
    print("Score on test data = ",score)
    print("ROCKET classifier score done!")
    return rocket_pipeline_classifier
    

In [179]:
def rocket_only(csv_directory_train, csv_directory_test, train_mfile, test_mfile, train_split_num, needed_columns, num_kernels):
    files_train = os.listdir(csv_directory_train)
    files_test = os.listdir(csv_directory_test)
    train_data = create_combined_data(csv_directory_train, files_train, needed_columns)
    test_data = create_combined_data(csv_directory_test, files_test, needed_columns)
    train_metrics = pd.read_csv(train_mfile)
    test_metrics = pd.read_csv(test_mfile)
    for d in train_data:
        print("Shape:", d.shape)
    rocket_partial = Rocket(num_kernels=num_kernels)
    rocket_partial.fit(train_data, train_metrics)
    return rocket_partial.transform(train_data)


In [184]:
split_num = 320
num_ops = 4
num_kernels = 1000

In [186]:
data_base = "/home/jharbin/academic/soprano/SPWorkerTemp/notebooks/predictor/temp-data/"
csv_directory_train = data_base + "/train"
csv_directory_test = data_base + "/test"
mfile_train = data_base + "metrics-train.csv"
mfile_test = data_base + "metrics-test.csv"

needed_columns = ['dosAttackTrigger', 'distortBaseScanPMB2', 'distortBaseScanOmni', 'distortBaseScanTiago', 'packetLossLaserScanOmni', 'packetLossLaserScanPMB2', 'packetLossLaserScanTiago', 'delayBaseScanOmni', 'delayBaseScanPMB2', 'delayBaseScanTiago1']
classifier = rocket_complete(csv_directory_train, csv_directory_test, mfile_train, mfile_test, split_num, needed_columns, num_kernels)
classifier
#test_data = create_combined_data(csv_directory_test, files_test, needed_columns)

Check data:  True
Check data:  True
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500, 10)
Shape: (500,

SklearnClassifierPipeline(classifier=RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])),
                          transformers=[Rocket(num_kernels=1000)])

In [193]:
csv_directory_train

'/home/jharbin/academic/soprano/SPWorkerTemp/notebooks/predictor/temp-data//train'

In [200]:
files_test = os.listdir(csv_directory_test)
test_data = create_combined_data(csv_directory_test, files_test, needed_columns)
zzz = [test_data[0]]
check_raise(zzz, mtype="df-list")

Check data:  True


True

In [284]:
test_num = 80

classifier.predict([test_data[test_num]])
#metric_test[]

IndexError: list index out of range